In [ ]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
import os.path
dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
dir_data_clim='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/clim/'
def sss_filename(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    fname_tem=syr + '/'+ 'mercatorglorys12v1_gl12_mean_' + syr + smon + '.nc'
    filename = dir_data + fname_tem
    return filename
def sss_filename_new(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    fname_tem=syr + '/'+ 'subset_mercatorglorys12v1_gl12_mean_' + syr + smon + '.nc'
    filename = dir_data + fname_tem
    return filename
def sss_filename_new_yearly(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    fname_tem=syr + '/'+ 'year_subset_mercatorglorys12v1_gl12_mean_' + syr + '.nc'
    filename = dir_data + fname_tem
    return filename

#get 0.25 km grid from cmc data to downsamplt this
filename = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/1994/002/19940102120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])
ds_v2.close()

In [ ]:
#files are huge becauseo of depth layers, so lower spatial resolution and only save surface data
for lyr in range(2004,2005): #2017):
    for imon in range(1,13): #3,13):
        d = dt.date(lyr,imon,1) 
        filename = sss_filename(d)
        if os.path.exists(filename):      
            ds = xr.open_dataset(filename)
            ds_subset = ds.sel(depth = ds.depth[0].data)
            ds_low_res = ds_subset.interp(latitude = new_lat,longitude = new_lon)
            ds.close()
            filename_new = sss_filename_new(d)
            ds_low_res.to_netcdf(filename_new)
            os.remove(filename)

In [ ]:
#make cmc monthly average ssts into one yearly file
for lyr in range(1993,2017): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        d = dt.date(lyr,imon,1) 
        filename = sss_filename_new(d)
        filename_new = sss_filename_new_yearly(d)
        ds = xr.open_dataset(filename)
        ds.close()
        ds_mnth.append(ds_low_res)
    combined = xr.concat(ds_mnth, dim='time')
    combined.to_netcdf(filename_new)
    

In [ ]:
#make climatology from monthly files
#make cmc climatology from monthly files
init = 0
for lyr in range(1993,1999): #2018): 
    d = dt.date(lyr,1,1) 
    filename = sss_filename_new_yearly(d)
    ds = xr.open_dataset(filename,drop_variables = ['usi','vsi','mlotst','zos','bottomT','sithick','siconc','thetao','uo','vo'])
    if init==0:
        ds_sum = ds
        init = 1
    else:
        ds_sum = xr.concat([ds_sum,ds],dim = 'time')
    ds.close()
    print(lyr,ds_sum.dims)
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='climatology_monthly_1993_1998_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum.to_netcdf(filename_out)
init = 0
for lyr in range(1999,2005): #2018): 
    d = dt.date(lyr,1,1) 
    filename = sss_filename_new_yearly(d)
    ds = xr.open_dataset(filename,drop_variables = ['usi','vsi','mlotst','zos','bottomT','sithick','siconc','thetao','uo','vo'])
    if init==0:
        ds_sum = ds
        init = 1
    else:
        ds_sum = xr.concat([ds_sum,ds],dim = 'time')
    ds.close()
    print(lyr,ds_sum.dims)
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='climatology_monthly_1999_2004_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum.to_netcdf(filename_out)
init = 0
for lyr in range(2005,2011): #2018): 
    d = dt.date(lyr,1,1) 
    filename = sss_filename_new_yearly(d)
    ds = xr.open_dataset(filename,drop_variables = ['usi','vsi','mlotst','zos','bottomT','sithick','siconc','thetao','uo','vo'])
    if init==0:
        ds_sum = ds
        init = 1
    else:
        ds_sum = xr.concat([ds_sum,ds],dim = 'time')
    ds.close()
    print(lyr,ds_sum.dims)
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='climatology_monthly_2005_2010_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum.to_netcdf(filename_out)
init = 0
for lyr in range(2011,2017): #2018): 
    d = dt.date(lyr,1,1) 
    filename = sss_filename_new_yearly(d)
    ds = xr.open_dataset(filename,drop_variables = ['usi','vsi','mlotst','zos','bottomT','sithick','siconc','thetao','uo','vo'])
    if init==0:
        ds_sum = ds
        init = 1
    else:
        ds_sum = xr.concat([ds_sum,ds],dim = 'time')
    ds.close()
    print(lyr,ds_sum.dims)
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='climatology_monthly_2011_2016_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum.to_netcdf(filename_out)
    

In [ ]:
#put clim together into one 
fname_tem='climatology_monthly_1993_1998_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = ds
ds.close()
fname_tem='climatology_monthly_1999_2004_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'time')
ds.close()
fname_tem='climatology_monthly_2005_2010_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'time')
ds.close()
fname_tem='climatology_monthly_2011_2016_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'time')
ds.close()
ds_sum = ds_sum.mean('time',skipna=True)
fname_tem='climatology_monthly_1993_2016_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum.to_netcdf(filename_out)


In [ ]:
ds_sum

In [ ]:
ds_sum